## Create static files for hydroflow

In [1]:
# The cells in this notebook run each step in the HydroFlow workflow
# This workflow was developed to function from within the designated SnowModel
# folder for each domain.

# Import all of the python packages used in this workflow.
from datetime import date, datetime
import json
import os
import requests

# Choose a domain
domain = 'YUKO'

# paths 
SMpath = '/nfs/attic/dfh/2020_NPRB/domain_'+domain+'/snowmodel2023_cfsv2/'
SMdatapath = '/nfs/attic/dfh/2020_NPRB/data/SMinputs/'+domain+'/'
# mkgdatpath = SMdatapath+'mk_grads_from_ascii_topoveg.f'
# topovegfile = 'dem_veg_'+domain+'.gdat'
# directionspath = SMdatapath+'direction/'
# watershedpath = SMdatapath+'watershed/'

#path to NPRB domains
domains_resp = requests.get("https://raw.githubusercontent.com/NPRB/02_preprocess_python/main/NPRB_domains.json")
domains = domains_resp.json()
    
# domain parameters
nx = domains[domain]['ncols']
ny = domains[domain]['nrows']
clsz = domains[domain]['cellsize']
xll = domains[domain]['xll']
yll = domains[domain]['yll']
st_dt = '2011-10-01'

## Define functions

In [3]:
# Function to edit fortran files
def replace_line(file_name, line_num, text):
    lines = open(file_name, 'r').readlines()
    lines[line_num] = text
    out = open(file_name, 'w')
    out.writelines(lines)
    out.close()

In [4]:
# Function to write out .ctl file for hydroflow steps
def disc_ctl(var,path,nx,ny,xll,yll,stdt,nt=1):
    '''
    vars can be:
    disc = hydroflow discharge
    tc = time coefficients
    watershed = watershed
    dir = flow direction 
    '''
    dt = datetime.strptime(st_dt, '%Y-%m-%d').strftime('%M:%SZ%d%b%Y')
    if var == 'disc':
        # Capture some variables from this SnowModel simulation
        lines = ['DSET ^disc.gdat',\
                 'UNDEF -9999.0',\
                 'OPTIONS BINPRECISION float32',\
                 'XDEF '+nx+' LINEAR '+xll+' '+clsz,\
                 'YDEF '+ny+' LINEAR '+yll+' '+clsz,\
                 'ZDEF 1 LEVELS 1',\
                 'TDEF '+str(nt)+' LINEAR '+dt+' 1dy',\
                 'VARS 2',\
                 'slow 1 0 SLOW',\
                 'fast 1 0 FAST',\
                 'ENDVARS']
    elif var == 'tc':
        lines = ['DSET ^tc.gdat',\
                 'UNDEF -9999.0',\
                 'OPTIONS BINPRECISION float32',\
                 'XDEF '+nx+' LINEAR '+xll+' '+clsz,\
                 'YDEF '+ny+' LINEAR '+yll+' '+clsz,\
                 'ZDEF 1 LEVELS 1',\
                 'TDEF '+str(nt)+' LINEAR '+dt+' 1dy',\
                 'VARS 2',\
                 'tcoef_slow 1 0 TCSLOW',\
                 'tcoef_fast 1 0 TCFAST',\
                 'ENDVARS']
    elif var == 'watershed':
        lines = ['DSET ^watershed.gdat',\
                 'UNDEF -9999.0',\
                 'OPTIONS BINPRECISION float32',\
                 'XDEF '+nx+' LINEAR '+xll+' '+clsz,\
                 'YDEF '+ny+' LINEAR '+yll+' '+clsz,\
                 'ZDEF 1 LEVELS 1',\
                 'TDEF '+str(nt)+' LINEAR '+dt+' 1dy',\
                 'VARS 3',\
                 'dir 1 0 flow direction',\
                 'wshed 1 0 watersheds',\
                 'order 1 0 flow accumulation',\
                 'ENDVARS']
    elif var == 'dir':
        lines = ['DSET ^dir.gdat',\
                 'UNDEF -9999.0',\
                 'OPTIONS BINPRECISION float32',\
                 'XDEF '+nx+' LINEAR '+xll+' '+clsz,\
                 'YDEF '+ny+' LINEAR '+yll+' '+clsz,\
                 'ZDEF 1 LEVELS 1',\
                 'TDEF '+str(nt)+' LINEAR '+dt+' 1dy',\
                 'VARS 1',\
                 'dir 1 0 flow direction',\
                 'ENDVARS']       

    with open(path, 'w') as f:
        for line in lines:
            f.write(line)
            f.write('\n')

    f.close() 

## Build combined dem and veg gdat & ctl files

Inputs: 
* [domain]+'_veg.asc'
* [domain]+'_dem.asc'

Outputs:

* dem_veg_[domain].gdat
* dem_veg_[domain].ctl

In [5]:
# Path to dem and veg files
dem_fname = domain+'_demHF.asc'
veg_fname = domain+'_vegHF.asc'
demveg_fname = 'dem_veg_'+domain+'.gdat'
demveg_ctl = 'dem_veg_'+domain+'.ctl'
topoveg_path = SMpath+'topo_vege/'

# Path to the flow directions fortran file for editing
mkgdat_fortran = SMdatapath+'mk_grads_from_ascii_topoveg.f'
print(mkgdat_fortran)

# edit the .f file for the domain
replace_line(mkgdat_fortran, 7, '      parameter (nx='+nx+',ny='+ny+')\n')
replace_line(mkgdat_fortran, 12, '      open (21,file=\''+dem_fname+'\')\n')
replace_line(mkgdat_fortran, 13, '      open (22,file=\''+veg_fname+'\')\n')
replace_line(mkgdat_fortran, 26, '      open (31,file=\''+demveg_fname+'\',\n')


/nfs/attic/dfh/2020_NPRB/data/SMinputs/YUKO/mk_grads_from_ascii_topoveg.f


In [6]:
# execute the function
# Use line magic to change directories to the file locations
# Run the fortran script and save the output
%cd $SMdatapath
!gfortran mk_grads_from_ascii_topoveg.f
!./a.out

/nfs/attic/dfh/2020_NPRB/data/SMinputs/YUKO


In [7]:
# write out corresponding .ctl file
ctl_list = ['DSET ^'+demveg_fname,
            'TITLE xxxxxxxxxxxxxxxxxxxxxxxxx',
            'UNDEF  -9999.0',
            'XDEF   '+nx+' LINEAR '+xll+' '+clsz,
            'YDEF   '+ny+' LINEAR '+yll+' '+clsz,
            'ZDEF    1 LEVELS 1',
            'TDEF    1 LINEAR 00Z01sep2001 1dy',
            'VARS    2',
            'topo    0  0 topo(m)',
            'veg     0  0 veg type',
            'ENDVARS']
with open(demveg_ctl, 'w') as f:
    f.write('\n'.join(ctl_list))

In [8]:
# move files to modeling space 
!mv $demveg_fname $topoveg_path
!mv $demveg_ctl $topoveg_path

### Create flow directions output file

Inputs: 
* dem_veg_[domain].gdat

Outputs:

* dir_[domain].gdat


In [9]:
# Paths to input/output file
demveg_fname = 'dem_veg_'+domain+'.gdat'
dir_fname = 'dir.gdat'
dir_ctl = 'dir.ctl' 

# Path to directions folder
dirs_path = SMpath+'direction/'

# Path to the flow directions fortran file for editing
dirs_fortran = SMpath+'direction/get_flow_dirs.f'
print(dirs_fortran)

# Run the replace line function in order to change the name of the input 
# and output files in the script
replace_line(dirs_fortran, 6, '      nx = '+nx+'\n')
replace_line(dirs_fortran, 7, '      ny = '+ny+'\n')
replace_line(dirs_fortran, 15, '      demfile = \'../topo_vege/'+demveg_fname+'\'\n')
replace_line(dirs_fortran, 19, '      dirfile = \''+dir_fname+'\'\n')

# Use line magic to change directories to the file locations
# Run the fortran script and save the output
%cd $dirs_path
!gfortran get_flow_dirs.f
!./a.out

#print .ctl file
ctlpath = dirs_path+'dir.ctl'
disc_ctl('dir',ctlpath,nx,ny,xll,yll,st_dt)

/nfs/attic/dfh/2020_NPRB/domain_YUKO/snowmodel2023_cfsv2/direction/get_flow_dirs.f
/nfs/attic/dfh/2020_NPRB/domain_YUKO/snowmodel2023_cfsv2/direction
get_flow_dirs.f:1084.72:

      IF(J.GT.97.OR.J.LT.1) PAUSE                                       
                                                                        1

N Rows =  527 N Columns =  863 N Nodes =  454801

COMPUTING PRIMARY FLOW DIRECTIONS

   Downward pass  1 FIRSTL =     1 LASTL =   527
   Downward pass  2 FIRSTL =     1 LASTL =   527
   Downward pass  3 FIRSTL =     1 LASTL =   527
   Downward pass  4 FIRSTL =     1 LASTL =   527
   Downward pass  5 FIRSTL =     1 LASTL =   527
   Downward pass  6 FIRSTL =     1 LASTL =   527
   Downward pass  7 FIRSTL =     1 LASTL =   527
   Downward pass  8 FIRSTL =     1 LASTL =   527
   Downward pass  9 FIRSTL =     1 LASTL =   527
   Downward pass 10 FIRSTL =     1 LASTL =   527
   Downward pass 11 FIRSTL =     1 LASTL =   527
   Downward pass 12 FIRSTL =     1 LASTL =   527
   

In [19]:
# # write out corresponding .ctl file
# ctl_list = ['DSET ^'+dir_fname,
#             'TITLE xxxxxxxxxxxxxxxxxxxxxxxxx',
#             'UNDEF  -9999.0',
#             'XDEF   '+nx+' LINEAR '+xll+' '+clsz,
#             'YDEF   '+ny+' LINEAR '+yll+' '+clsz,
#             'ZDEF    1 LEVELS 1',
#             'TDEF    1 LINEAR 00Z01sep2001 1dy',
#             'VARS    1',
#             'dir 1 0 flow direction',
#             'ENDVARS']

# with open(dir_ctl, 'w') as f:
#     f.write('\n'.join(ctl_list))

### Create the Watersheds output file

Inputs: 
* dem_veg_[domain].gdat
* dir_[domain].gdat

Outputs:

*  watersheds.gdat
* watersheds_index.txt

In [20]:
# # Paths to input/output file
# demveg_fname = 'dem_veg_'+domain+'.gdat'
# dir_fname = 'dir.gdat'
# wshed_fname = 'watershed_upslope.gdat'
# wshed_ctl = 'watershed_upslope.ctl'
# wshedidx_fname = 'watershed_index_upslope.txt'

# # Path to watersheds folder
# wshed_path = SMpath+'watershed/'

# # Path to the watersheds fortran file for editing
# wshed_fortran = SMpath+'watershed/mk_watersheds.f'
# print(wshed_fortran)

# # Run the replace line function in order to change the name of the input 
# # and output files in the script

# replace_line(wshed_fortran, 11, '      parameter (nx='+nx+',ny='+ny+')\n')
# replace_line(wshed_fortran, 34, '      open (unit=51,file=\'../direction/'+dir_fname+'\',\n')
# replace_line(wshed_fortran, 39, '      open (unit=61,file=\'../topo_vege/'+demveg_fname+'\',\n')
# replace_line(wshed_fortran, 65, '      open (unit=41,file=\''+wshedidx_fname+'\')\n')
# replace_line(wshed_fortran, 73, '      open (unit=61,file=\''+wshed_fname+'\',\n')

# # Use line magic to change directories to the file locations
# # Run the fortran script and save the output
# %cd $wshed_path
# !gfortran mk_watersheds.f
# !./a.out

# #print .ctl file
# ctlpath = wshed_path+wshed_ctl
# disc_ctl('watershed',ctlpath,nx,ny,xll,yll,st_dt)

/nfs/attic/dfh/2020_NPRB/domain_YUKO/snowmodel2023_cfsv2/watershed/mk_watersheds.f
/nfs/attic/dfh/2020_NPRB/domain_YUKO/snowmodel2023_cfsv2/watershed

 calculating grid cell connectivity

 identifying watersheds

^C


In [10]:
# Paths to input/output file
demveg_fname = 'dem_veg_'+domain+'.gdat'
dir_fname = 'dir.gdat'
wshed_fname = 'watershed.gdat'
wshed_ctl = 'watershed.ctl'
wshedidx_fname = 'watershed_index.txt'

# Path to watersheds folder
wshed_path = SMpath+'watershed/'

# Path to the watersheds fortran file for editing
wshed_fortran = SMpath+'watershed/mk_watersheds.f'
print(wshed_fortran)

# Run the replace line function in order to change the name of the input 
# and output files in the script

replace_line(wshed_fortran, 11, '      parameter (nx='+nx+',ny='+ny+')\n')
replace_line(wshed_fortran, 34, '      open (unit=51,file=\'../direction/'+dir_fname+'\',\n')
replace_line(wshed_fortran, 39, '      open (unit=61,file=\'../topo_vege/'+demveg_fname+'\',\n')
replace_line(wshed_fortran, 65, '      open (unit=41,file=\''+wshedidx_fname+'\')\n')
replace_line(wshed_fortran, 73, '      open (unit=61,file=\''+wshed_fname+'\',\n')

# Use line magic to change directories to the file locations
# Run the fortran script and save the output
%cd $wshed_path
!gfortran mk_watersheds.f
!./a.out

#print .ctl file
ctlpath = wshed_path+wshed_ctl
disc_ctl('watershed',ctlpath,nx,ny,xll,yll,st_dt)

/nfs/attic/dfh/2020_NPRB/domain_YUKO/snowmodel2023_cfsv2/watershed/mk_watersheds.f
/nfs/attic/dfh/2020_NPRB/domain_YUKO/snowmodel2023_cfsv2/watershed

 calculating grid cell connectivity

 identifying watersheds

 defining watershed connectivity

 calculating ODE numbers and positions
      1) working on watershed number        1000
      1) working on watershed number        2000
      1) working on watershed number        3000
      1) working on watershed number        4000

 creating the connectivity index

 listing the connectivity information in sequential order
      2) working on watershed number        1000
      2) working on watershed number        2000
      2) working on watershed number        3000
      2) working on watershed number        4000

 defining the solution order of the watershed grid cells
      3) working on watershed number        1000
      3) working on watershed number        2000
      3) working on watershed number        3000
      3) working on wate

In [33]:
# # write out corresponding .ctl file
# ctl_list = ['DSET ^'+wshed_fname,
#             'TITLE xxxxxxxxxxxxxxxxxxxxxxxxx',
#             'UNDEF  -9999.0',
#             'XDEF   '+nx+' LINEAR '+xll+' '+clsz,
#             'YDEF   '+ny+' LINEAR '+yll+' '+clsz,
#             'ZDEF    1 LEVELS 1',
#             'TDEF    1 LINEAR 00Z01sep2001 1dy',
#             'VARS    3',
#             'dir 1 0 flow direction',
#             'wshed 1 0 watersheds',
#             'order 1 0 flow accumulation',
#             'ENDVARS']

# with open(wshed_ctl, 'w') as f:
#     f.write('\n'.join(ctl_list))